In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [19]:
keys = pd.read_csv('./011edlcredit.csv')

In [12]:
url = "https://pitt.sona-systems.com/"
email = '***'
passw = '***'
study_name = 'Emotions in Daily Life'

In [7]:
def transform(name):
    '''
    takes names as given from the other script and cleans them
    '''
    name_s = name.split(',')
    new_name = name_s[1] + ' ' + name_s[0]
    return(new_name.lstrip())

In [15]:
def sonaLogin(email = email, passw = passw):
    '''
    INPUT: user credentials
    
    OUTPUT: webdriver
    '''
    driver = webdriver.Chrome()
    driver.get(url)

    username = driver.find_element_by_xpath("/html/body/form/div[3]/section/section/section/div[2]/div[1]/section/div/div/div/div/input[1]")
    username.send_keys(email)

    password =  driver.find_element_by_xpath("/html/body/form/div[3]/section/section/section/div[2]/div[1]/section/div/div/div/div/input[2]")
    password.send_keys(passw)

    submit_btn = driver.find_element_by_xpath("/html/body/form/div[3]/section/section/section/div[2]/div[1]/section/div/div/div/div/input[3]")
    submit_btn.click()

    to_credit = driver.find_element_by_xpath("/html/body/form/div[3]/section/section/section/div[2]/div[1]/section[1]/div/a[3]")
    to_credit.click()
    
    return(driver)
    

In [25]:
def give_credit(driver, study_name):
    '''
    INPUT: study_name, webdriver
    
    OUTPUT: none, makes edits to sona credit options
    
    iterates through table, assesses if it matches study, locates name cell, checks table to see if credit updates,
    then marks the appropriate box
    
    Submit is not automated to allow for spot checks
    '''
    credits = driver.find_element_by_css_selector('#no-more-tables > table')

#iterate through participants
    row_num = 0
    for row in credits.find_elements_by_css_selector('tr'):
        isSBDL = False
        keep_next = False
        checked_credit = False
        gets_credit = False
    
    #iterate through row
        for cell in row.find_elements_by_tag_name('td'):
        #make sure it is sbdl
            if cell.text == study_name:
                isSBDL = True
                print('isSBDL')
        #placeholder
            elif 'Online study' in cell.text and isSBDL == True:
                keep_next = True
        #check credit
            elif keep_next == True:
            
                key = cell.text
                print(key + ' NOW SEARCHING')
                part_row = keys[keys['cleanname'] == key]
            
                if len(part_row) > 0:
                    print(key + ' FOUND')
                    if part_row['credit'].mean() == 1:
                        gets_credit = True
                    checked_credit = True
                else:
                    print(key + ' NOT FOUND')
            
            
                keep_next = False
            
            elif checked_credit == True:
                time.sleep(2)
                if gets_credit == True:
                    grant_credit = driver.find_element_by_xpath('//tr[' + str(row_num) + ']/td[5]/div/label[2]/label')
                    print(key + ' GRANT')
                    time.sleep(1)
                    grant_credit.click()
                else:
                    time.sleep(1)
                    ens = driver.find_element_by_xpath('//tr[' + str(row_num) + ']/td[5]/div/label[4]/label')
                    print(key + ' ENS')
                    ens.click()
        row_num += 1
            

In [21]:
def full_path_credit(keys = keys, study_name = study_name, email = email, passw = passw):
    keys['cleanname'] = keys['name'].apply(transform)
    driver = sonaLogin(email,passw)
    give_credit(driver, study_name)

In [ ]:
full_path_credit()